# ids projekti, kivaa :---)

tästä alkaa suuri seikkailu. Olkaamme innokaita ja tehokkaita. 

In [1]:
import json
import pandas as pd
import numpy as np
import urllib.request as ur
import requests
from datetime import timedelta, date
from sklearn.model_selection import train_test_split as tts
from tpot import TPOTClassifier

In [2]:

def date_range(n):
    date_list = []

    for single_date in (date.today() - timedelta(n+1) for n in range(n)):
        date_list.append(single_date.strftime('%Y%m%d'))

    return(date_list)

In [12]:

def fetch_game_scoreboards():
    games_lists = []

    auth=('Enzine', 'Enzine')

    url = 'https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/scoreboard.json?fordate={date}'
    
    DATE_RANGE = 100
    n=0
    for i in date_range(DATE_RANGE):
        if n%50 == 0:
            print(n)
            
        n += 1
        
        try:
            json = requests.get(url.format(date=i), auth=auth).json()
            json['scoreboard']['gameScore']
        except:
            continue
        
        games_lists.append(json['scoreboard']['gameScore'])
        
       

    return(games_lists)

In [13]:
def fetch_team_data():
    auth=('Enzine', 'Enzine')
    url = 'https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/overall_team_standings.json'
    
    json = requests.get(url, auth=auth).json()
    
    data = json['overallteamstandings']['teamstandingsentry']
    
    return(data)

In [11]:
#LATAA PELIT
#TÄSSÄ KESTÄÄ KAUAN, KANNATTAA TEHDÄ VAIN KERRAN

game_data = np.hstack(fetch_game_scoreboards())

print(game_data.shape)

0
(135,)


In [55]:
team_data = np.hstack(fetch_team_data())

print(team_data.shape)

(31,)


In [40]:
#Jos kotijoukkoe voittaa, winner = 1

print(team_data[0])

{'team': {'ID': '1', 'City': 'Tampa Bay', 'Name': 'Lightning', 'Abbreviation': 'TBL'}, 'rank': '1', 'stats': {'GamesPlayed': {'@abbreviation': 'GP', '#text': '10'}, 'stats': {'Wins': {'@abbreviation': 'W', '#text': '8'}, 'Losses': {'@abbreviation': 'L', '#text': '1'}, 'OvertimeWins': {'@abbreviation': 'OTW', '#text': '0'}, 'OvertimeLosses': {'@abbreviation': 'OTL', '#text': '1'}, 'GoalsFor': {'@abbreviation': 'GF', '#text': '41'}, 'GoalsAgainst': {'@abbreviation': 'GA', '#text': '24'}, 'FaceoffWins': {'@abbreviation': 'F/O W', '#text': '309'}, 'FaceoffLosses': {'@abbreviation': 'F/O L', '#text': '329'}, 'FaceoffPercent': {'@abbreviation': 'F/O %', '#text': '48.4'}, 'Powerplays': {'@abbreviation': 'PP', '#text': '41'}, 'PowerplayGoals': {'@abbreviation': 'PPG', '#text': '12'}, 'PowerplayPercent': {'@abbreviation': 'PP%', '#text': '29.3'}, 'PenaltyKills': {'@abbreviation': 'PK', '#text': '39'}, 'PenaltyKillGoalsAllowed': {'@abbreviation': 'PKGA', '#text': '7'}, 'PenaltyKillPercent': {'@a

In [125]:
from pandas.io.json import json_normalize

def fix_team_data():
    df = json_normalize(team_data)

    #test.drop(['*.@abbreviation'], axis=1, inplace=True)

    df = df[df.columns.drop(list(df.filter(regex='abbreviation|Abbreviation')))]

    df.drop(['team.Name', 'team.City'], axis=1, inplace=True)

    return(df)

team_standings = fix_team_data()


In [180]:
id_to_teamname = dict()

def parse_game(g):
    data = dict()
    
    # DEFINE IDS
    away_team_id = g['game']['awayTeam']['ID']
    home_team_id = g['game']['homeTeam']['ID']
    
    
    data['away_team_id'] = away_team_id
    data['home_team_id'] = home_team_id
    
    
    #ASSOCIATE TEAMNAMES WITH TEAM IDS
    if g['game']['awayTeam']['Abbreviation'] not in id_to_teamname:
        id_to_teamname[g['game']['awayTeam']['Abbreviation']] = g['game']['awayTeam']['ID']
        
    if g['game']['homeTeam']['Abbreviation'] not in id_to_teamname:
        id_to_teamname[g['game']['homeTeam']['Abbreviation']] = g['game']['homeTeam']['ID']
    
    
    # DEFINE WINNER
    if g['homeScore'] > g['awayScore']:
        data['winner'] = 1
    else:
        data['winner'] = 0
        
    #data['away_team_stats'] = team_standings.loc[(team_standings['team.ID'] == away_team_id)].values.flatten()
    #data['home_team_stats'] = team_standings.loc[(team_standings['team.ID'] == home_team_id)].values.flatten()

    return(pd.Series(data))


In [181]:
match_data = pd.DataFrame()


for g in game_data:
    match_data = match_data.append(parse_game(g), ignore_index=True)

print(match_data.shape)

(135, 3)


In [238]:
match_data.drop('winner', axis=1).values

array([['30', '8'],
       ['24', '10'],
       ['29', '6'],
       ['1', '3'],
       ['16', '15'],
       ['28', '13'],
       ['4', '14'],
       ['23', '18'],
       ['21', '25'],
       ['27', '22'],
       ['20', '142'],
       ['28', '12'],
       ['26', '9'],
       ['21', '16'],
       ['18', '9'],
       ['24', '6'],
       ['12', '13'],
       ['10', '1'],
       ['26', '8'],
       ['15', '11'],
       ['28', '19'],
       ['4', '5'],
       ['3', '27'],
       ['20', '30'],
       ['25', '23'],
       ['17', '142'],
       ['21', '15'],
       ['26', '7'],
       ['10', '4'],
       ['5', '16'],
       ['25', '47'],
       ['14', '29'],
       ['18', '6'],
       ['1', '19'],
       ['8', '9'],
       ['21', '11'],
       ['7', '13'],
       ['24', '20'],
       ['3', '23'],
       ['17', '22'],
       ['27', '30'],
       ['16', '12'],
       ['20', '17'],
       ['14', '28'],
       ['12', '5'],
       ['4', '6'],
       ['10', '9'],
       ['1', '7'],
       ['21', '13'

In [244]:
#team_standings.loc[(team_standings['team.ID'] == '1')]

great_match_arrays = []

d = match_data.drop('winner', axis=1).values

for i in d:
    home = i[1]
    away = i[0]
    great_match_arrays.append(np.hstack([i, team_standings.loc[(team_standings['team.ID'] == home)].values.flatten(), team_standings.loc[(team_standings['team.ID'] == away)].values.flatten()]))
    
#b = match_data.values[0]
#a = team_standings.loc[(team_standings['team.ID'] == '30')].values.flatten()
#c = team_standings.loc[(team_standings['team.ID'] == '8')].values.flatten()

final_data = np.stack(great_match_arrays)

print(final_data.shape)

(135, 52)


In [245]:
target = match_data['winner'].values

X_train, X_test, y_train, y_test = tts(final_data, target, train_size=0.75, test_size=0.25, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(101, 52) (34, 52) (101,) (34,)


In [246]:

tpot = TPOTClassifier(generations=25, population_size=25, verbosity=2)
tpot.fit(X_train, y_train)


Optimization Progress:   8%|▊         | 50/650 [00:13<04:53,  2.04pipeline/s]

Generation 1 - Current best internal CV score: 0.7025563909774436


Optimization Progress:  12%|█▏        | 75/650 [00:50<06:30,  1.47pipeline/s]

Generation 2 - Current best internal CV score: 0.7025563909774436


Optimization Progress:  15%|█▌        | 100/650 [01:10<09:29,  1.03s/pipeline]

...

Generation 23 - Current best internal CV score: 0.7130827067669173


Optimization Progress:  96%|█████████▌| 625/650 [10:40<00:44,  1.78s/pipeline]

Generation 24 - Current best internal CV score: 0.7130827067669173


Generation 25 - Current best internal CV score: 0.7130827067669173

Best pipeline: ExtraTreesClassifier(GaussianNB(RFE(input_matrix, criterion=gini, max_features=0.55, n_estimators=100, step=0.35)), bootstrap=False, criterion=entropy, max_features=0.35, min_samples_leaf=6, min_samples_split=4, n_estimators=100)


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT....45,
        0.5 ,  0.55,  0.6 ,  0.65,  0.7 ,  0.75,  0.8 ,  0.85,  0.9 ,
        0.95,  1.  ])}}}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=25, max_eval_time_mins=5,
        max_time_mins=None, mutation_rate=0.9, n_jobs=1, offspring_size=25,
        periodic_checkpoint_folder=None, population_size=25,
        random_state=None, scoring=None, subsample=1.0, verbosity=2,
        warm_start=False)

In [249]:

print(tpot.score(X_test, y_test))
#tpot.export('ExtraTreesClassifier001.py')





0.676470588235


True

In [286]:
#print(id_to_teamname)

def predict_game(away, home):
    away_id = id_to_teamname[away]
    home_id = id_to_teamname[home]
    
    away_stats = team_standings.loc[(team_standings['team.ID'] == away_id)].values.flatten()
    home_stats = team_standings.loc[(team_standings['team.ID'] == home_id)].values.flatten()
    
    vec = np.hstack([away_id, home_id, home_stats, away_stats]).reshape(1, -1)

    res = tpot.predict(vec.reshape(1, -1))
    
    if res == 1:
        return(home)
    else:
        return(away)


In [288]:
predict_game('BUF', 'COL')

#print(id_to_teamname)

'COL'

###### 25.10.17

CAL - STL --- STL VOITTAA

BUF - COL --- COL VOITTAA